In [20]:
import site, pathlib, subprocess, sys
sys.path.insert(0,"/home/mt622/GitHubRepos/FractionalBrownianMotion")
repo_root = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], text=True
).strip()
pth_dir = pathlib.Path(site.getsitepackages()[0])
(pth_dir / "namerepo_root.pth").write_text(repo_root + "\n")
print("Wrote", pth_dir / "namerepo_root.pth")

Wrote /Users/marcos/GitHubRepos/FractionalBrownianMotion/venv/lib/python3.10/site-packages/namerepo_root.pth


In [21]:
from multiprocessing import shared_memory
from src.classes.ClassFractionalLorenz96 import FractionalLorenz96
import math
import numpy as np
import torch
import os
from tqdm import tqdm
from src.generative_modelling.models.ClassVPSDEDiffusion import VPSDEDiffusion
from src.generative_modelling.models.TimeDependentScoreNetworks.ClassConditionalMarkovianTSPostMeanScoreMatching import \
    ConditionalMarkovianTSPostMeanScoreMatching

In [22]:
# Data parameters
from configs.RecursiveVPSDE.Markovian_40DLorenz.recursive_Markovian_PostMeanScore_40DLorenz_Chaos_T256_H05_tl_110data_StbleTgt import get_config
config = get_config()

In [23]:
assert (config.hurst == 0.5)
assert (config.early_stop_idx == 0)
assert (config.tdata_mult == 110)
#assert (config.sin_space_scale == 25.)
assert (config.feat_thresh != 1./1.)
print(config.scoreNet_trained_path, config.dataSize)
rng = np.random.default_rng()
diffusion = VPSDEDiffusion(beta_max=config.beta_max, beta_min=config.beta_min)


/Users/marcos/GitHubRepos/FractionalBrownianMotion/src/generative_modelling/trained_models/trained_rec_PM_ST_0100FTh_MLP_2LFac_NSTgtNFMReg_40DLnz_125e+00FConst_VPSDE_T256_Ndiff10000_Tdiff1000e+00_DiffEmbSz64_ResLay10_ResChan8_DiffHdnSz64_TrueHybd_TrueWghts_t00_dT3906e-03_MLP_H4_CUp20_tl110 40000


In [24]:
model_dir = "/".join(config.scoreNet_trained_path.split("/")[:-1]) + "/"
entered = False
for file in os.listdir(model_dir):
    if config.scoreNet_trained_path in os.path.join(model_dir, file) and ("Trk" not in file):
        good = ConditionalMarkovianTSPostMeanScoreMatching(
    *config.model_parameters)
        print(file)
        entered = True
        good.load_state_dict(torch.load(os.path.join(model_dir, file)))
        try:
            best_epoch = int(file.split("Trk")[-1][3:])
        except ValueError as e:
            best_epoch = int(file.split("NEp")[-1])
assert entered
print(best_epoch)

trained_rec_PM_ST_0100FTh_MLP_2LFac_NSTgtNFMReg_40DLnz_125e+00FConst_VPSDE_T256_Ndiff10000_Tdiff1000e+00_DiffEmbSz64_ResLay10_ResChan8_DiffHdnSz64_TrueHybd_TrueWghts_t00_dT3906e-03_MLP_H4_CUp20_tl110_NEp3000
3000


In [25]:
def true_drift(prev, num_paths, config):
    assert (prev.shape == (num_paths, config.ndims))
    drift_X = np.zeros((num_paths, config.ndims))
    for i in range(config.ndims):
        drift_X[:, i] = (prev[:, (i + 1) % config.ndims] - prev[:, i - 2]) * prev[:, i - 1] - prev[:,
                                                                                              i] *config.forcing_const
    return drift_X

In [26]:
import torch.nn.functional as F
def experiment_MLP_1D_drifts(config, es,Xs, good, onlyGauss=False):
    if config.has_cuda:
        device = 0#int(os.environ["LOCAL_RANK"])
    else:
        device = torch.device("cpu")
    Xs = torch.Tensor(Xs).to(device)
    good = good.to(device)
    diffusion = VPSDEDiffusion(beta_max=config.beta_max, beta_min=config.beta_min)
    ts_step = config.deltaT
    Xshape = Xs.shape[0]
    num_taus = 10

    num_diff_times = config.max_diff_steps
    Ndiff_discretisation = config.max_diff_steps
    diffusion_times = torch.linspace(start=config.sample_eps, end=config.end_diff_time,
                                     steps=Ndiff_discretisation).to(device)

    features_tensor = torch.stack([Xs for _ in range(1)], dim=0).reshape(Xshape * 1, 1, -1).to(device)
    vec_Z_taus = diffusion.prior_sampling(shape=(Xshape * num_taus, 1, config.ts_dims)).to(device)

    # ts = []
    es = num_diff_times - es
    final_vec_mu_hats = np.zeros(
        (Xshape, es, num_taus, config.ts_dims))  # Xvalues, DiffTimes, Ztaus, Ts_Dims

    ts = []
    # mu_hats_mean = np.zeros((tot_num_feats, num_taus))
    # mu_hats_std = np.zeros((tot_num_feats, num_taus))
    good.eval()
    insert_idx=-1
    for difftime_idx in (np.arange(num_diff_times - 1, num_diff_times - es - 1, -1)): #difftime_idx >= num_diff_times - es:
        d = diffusion_times[Ndiff_discretisation - (num_diff_times - 1 - difftime_idx) - 1].to(device)
        diff_times = torch.stack([d for _ in range(Xshape)]).reshape(Xshape * 1).to(device)
        eff_times = diffusion.get_eff_times(diff_times=diff_times).unsqueeze(-1).unsqueeze(-1).to(device)
        vec_diff_times = torch.stack([diff_times for _ in range(num_taus)], dim=0).reshape(num_taus * Xshape)
        vec_eff_times = torch.stack([eff_times for _ in range(num_taus)], dim=0).reshape(num_taus * Xshape, 1, 1)
        vec_conditioner = torch.stack([features_tensor for _ in range(num_taus)], dim=0).reshape(
            num_taus * Xshape,
            1, -1)
        with torch.no_grad():
            if onlyGauss:
                scoreEval_vec_Z_taus = torch.randn_like(vec_Z_taus).to(device)
            else:
                scoreEval_vec_Z_taus = vec_Z_taus
            vec_predicted_score = good.forward(inputs=scoreEval_vec_Z_taus, times=vec_diff_times, conditioner=vec_conditioner,
                                             eff_times=vec_eff_times)

        beta_taus = torch.exp(-0.5 * eff_times[0, 0, 0]).to(device)
        sigma_taus = torch.pow(1. - torch.pow(beta_taus, 2), 0.5).to(device)
        sigma2_taus=torch.pow(1. - torch.pow(beta_taus, 2), 1.).to(device)
        predicted_score = -scoreEval_vec_Z_taus/sigma2_taus + (beta_taus/sigma2_taus)*vec_predicted_score
        vec_scores, vec_drift, vec_diffParam = diffusion.get_conditional_reverse_diffusion(x=vec_Z_taus,
                                                                                           predicted_score=predicted_score,
                                                                                           diff_index=torch.Tensor(
                                                                                               [int((
                                                                                                       num_diff_times - 1 - difftime_idx))]).to(
                                                                                               device),
                                                                                           max_diff_steps=Ndiff_discretisation)

        if "PM" in config.scoreNet_trained_path:
            final_mu_hats = (beta_taus*scoreEval_vec_Z_taus / (sigma2_taus)) + ((
                                                                            (torch.pow(sigma_taus, 2) + (
                                                                                    torch.pow(beta_taus * config.diffusion,
                                                                                            2) * ts_step)) / (
                                                                                    ts_step * sigma2_taus)) * vec_predicted_score)
        else:
            final_mu_hats = (scoreEval_vec_Z_taus / (ts_step * beta_taus)) + ((
                                                                            (sigma2_taus + (
                                                                                    torch.pow(beta_taus * config.diffusion,
                                                                                            2) * ts_step)) / (
                                                                                    ts_step * beta_taus)) * vec_scores)

        assert (final_mu_hats.shape == (num_taus * Xshape, 1, config.ts_dims))
        means = final_mu_hats.reshape((num_taus, Xshape, config.ts_dims))

        # print(vec_Z_taus.shape, vec_scores.shape)
        final_vec_mu_hats[:, insert_idx,:, :] = means.permute((1, 0, 2)).cpu().numpy()
        vec_z = torch.randn_like(vec_drift).to(device)
        vec_Z_taus = vec_drift + vec_diffParam * vec_z
        insert_idx -=1
    assert (final_vec_mu_hats.shape == (Xshape, es, num_taus, config.ts_dims))
    return final_vec_mu_hats

In [27]:
def _get_device(device_str: str | None = None):
    if device_str is not None:
        return torch.device(device_str)
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

@torch.no_grad()
def true_drift_gpu(prev: torch.Tensor, num_paths: int, config) -> torch.Tensor:
    """
    prev: (num_paths, d) float32 CUDA tensor
    returns: (num_paths, 1, d) float32 CUDA tensor
    """
    # Ensure shape
    assert prev.ndim == 2 and prev.shape[0] == num_paths and prev.shape[1] == config.ndims
    x = prev
    # Vectorized Lorenz96 drift:
    x_ip1 = torch.roll(x, shifts=-1, dims=1)  # x_{i+1}
    x_im1 = torch.roll(x, shifts=1,  dims=1)  # x_{i-1}
    x_im2 = torch.roll(x, shifts=2,  dims=1)  # x_{i-2}
    drift = (x_ip1 - x_im2) * x_im1 - x*float(config.forcing_const)
    return drift[:, None, :]


@torch.no_grad()
def IID_NW_multivar_estimator_gpu(
    prevPath_observations: torch.Tensor,  # (N,n,d) float32 CUDA
    path_incs: torch.Tensor,              # (N,n,d) float32 CUDA
    inv_H: torch.Tensor,                  # (d,) (diag) or (d,d) float32 CUDA
    norm_const: float,                    # same meaning as your CPU code
    x: torch.Tensor,                      # (M,d) float32 CUDA
    t1: float,
    t0: float,
    truncate: bool = True,
    M_tile: int = 32,                     # micro-batch states
    Nn_tile: int | None = 512_000,        # micro-batch samples (None => full)
    stable: bool = True,
) -> torch.Tensor:
    """
    Returns: (M,d) float32 CUDA tensor (keeps all heavy ops on LongerTimes_GPU).
    Matches your scaling:
      denom = sum(w)/(N*n)
      numer = (sum(w * incs)/N) * (t1 - t0)
    """
    #assert prevPath_observations.is_cuda and path_incs.is_cuda and x.is_cuda
    assert prevPath_observations.dtype == torch.float32
    assert path_incs.dtype == torch.float32
    assert x.dtype == torch.float32

    N, n, d = prevPath_observations.shape
    Nn = N * n
    if Nn_tile is None or Nn_tile > Nn:
        Nn_tile = Nn

    # Flatten once
    mu = prevPath_observations.reshape(Nn, d).contiguous()  # (Nn,d)
    dX = path_incs.reshape(Nn, d).contiguous()              # (Nn,d)

    # Diagonal vs full inv_H
    diag = (inv_H.ndim == 1)
    if diag:
        A = inv_H                                           # (d,)
        muAh = mu * A                                       # (Nn,d)
        mu_quad = (mu * muAh).sum(-1)                       # (Nn,)
        def xAh(X): return X * A
    else:
        A = inv_H                                           # (d,d)
        muAh = mu @ A                                       # (Nn,d)
        mu_quad = (mu * muAh).sum(-1)                       # (Nn,)
        # Sanity: PD
        sign, _ = torch.linalg.slogdet(A)
        if sign.item() <= 0:
            raise ValueError("inv_H must be positive definite.")

    # Use log(norm_const) directly to match your CPU estimator
    log_norm_const = float(np.log(norm_const))

    M = x.size(0)
    # Accumulate in float64 for stability
    denom = torch.zeros(M,     dtype=torch.float64, device=x.device)
    numer = torch.zeros(M, d,  dtype=torch.float64, device=x.device)

    for m0 in range(0, M, M_tile):
        X = x[m0:m0 + M_tile]                     # (mb,d)
        XAh = xAh(X)                               # (mb,d)
        X_quad = (X * XAh).sum(-1)                 # (mb,)

        denom_tile = torch.zeros(X.size(0),    dtype=torch.float64, device=x.device)
        numer_tile = torch.zeros(X.size(0), d, dtype=torch.float64, device=x.device)

        if stable:
            lse_max = torch.full((X.size(0),), -torch.inf, dtype=torch.float32, device=x.device)
            # First pass: find max exponent per state (over all Nn tiles)
            for i0 in range(0, Nn, Nn_tile):
                muq_i  = mu_quad[i0:i0 + Nn_tile]             # (bn,)
                muAh_i = muAh[i0:i0 + Nn_tile]                # (bn,d)
                cross  = muAh_i @ X.t()                       # (bn,mb)
                expo   = log_norm_const - 0.5 * (muq_i[:, None] + X_quad[None, :] - 2.0 * cross)
                lse_max = torch.maximum(lse_max, expo.max(dim=0).values)

        # Second pass: accumulate with stabilization (or plain)
        for i0 in range(0, Nn, Nn_tile):
            muAh_i = muAh[i0:i0 + Nn_tile]                    # (bn,d)
            muq_i  = mu_quad[i0:i0 + Nn_tile]                 # (bn,)
            dX_i   = dX[i0:i0 + Nn_tile]                      # (bn,d)

            cross = muAh_i @ X.t()                            # (bn,mb)
            expo  = log_norm_const - 0.5 * (muq_i[:, None] + X_quad[None, :] - 2.0 * cross)

            if stable:
                w = torch.exp(expo - lse_max[None, :])        # (bn,mb)
            else:
                w = torch.exp(expo)

            denom_tile += w.sum(dim=0, dtype=torch.float64) / (N * n)
            numer_tile += (w.t() @ dX_i).to(torch.float64) * ((t1 - t0) / N)

        if stable:
            scale = torch.exp(lse_max.to(torch.float64))
            denom_tile *= scale
            numer_tile *= scale[:, None]

        denom[m0:m0 + X.size(0)] += denom_tile
        numer[m0:m0 + X.size(0)] += numer_tile

    est = (numer / denom[:, None]).to(torch.float32)          # (M,d)

    if truncate:
        m = denom.min()
        est[denom <= (m / 2.0)] = 0

    return est

def run_Nadaraya(config, Xs):
    num_paths = 1024 if config.feat_thresh == 1. else 1024
    assert num_paths == 1024
    t0 = config.t0
    deltaT = config.deltaT
    t1 = deltaT * config.ts_length
    # Drift parameters
    diff = config.diffusion
    initial_state = np.array(config.initState)
    rvs = None
    H = config.hurst
    try:
        is_path_observations = np.load(config.data_path, allow_pickle=True)[:num_paths, :, :]
        is_path_observations = np.concatenate(
            [np.repeat(np.array(config.initState).reshape((1, 1, config.ndims)), is_path_observations.shape[0], axis=0),
             is_path_observations], axis=1)
        assert is_path_observations.shape == (num_paths, config.ts_length + 1, config.ndims)
    except (FileNotFoundError, AssertionError) as e:
        print(e)
        fLnz = FractionalLorenz96(X0=config.initState, diff=config.diffusion, num_dims=config.ndims,
                                  forcing_const=config.forcing_const)
        is_path_observations = np.array(
            [fLnz.euler_simulation(H=H, N=config.ts_length, deltaT=deltaT, X0=initial_state, Ms=None, gaussRvs=rvs,
                                   t0=t0, t1=t1) for _ in (range(num_paths))]).reshape(
            (num_paths, config.ts_length + 1, config.ndims))
        np.save(config.data_path, is_path_observations[:, 1:, :])
        assert is_path_observations.shape == (num_paths, config.ts_length + 1, config.ndims)

    is_idxs = np.arange(is_path_observations.shape[0])
    path_observations = is_path_observations[np.random.choice(is_idxs, size=num_paths, replace=False), :]
    # We note that we DO NOT evaluate the drift at time t_{0}=0
    # We therefore remove the first element of path_observations since it includes X_{t_{0}} = X_{0}
    # We also remove the last element since we never evaluate the drift at that point
    t0 = deltaT
    prevPath_observations = path_observations[:, 1:-1, :]
    # We compute the path incs with respect to the prevPath_observations (since X_{t_{0}} != X_{0})
    path_incs = np.diff(path_observations, axis=1)[:, 1:, :]
    assert (prevPath_observations.shape == path_incs.shape)
    assert (path_incs.shape[1] == config.ts_length - 1)
    assert (path_observations.shape[1] == prevPath_observations.shape[1] + 2)
    assert (prevPath_observations.shape[1] * deltaT == (t1 - t0))
    grid_1d = np.logspace(-3.55, -0.05, 30)
    xadd = np.logspace(-0.05, 1.0, 11)[1:]  # 10 values > -0.05
    grid_1d = np.concatenate([grid_1d, xadd])
    bws = np.stack([grid_1d for m in range(config.ndims)], axis=-1)
    assert (bws.shape == (40, config.ndims))

    prevPath_shm = shared_memory.SharedMemory(create=True, size=prevPath_observations.nbytes)
    path_incs_shm = shared_memory.SharedMemory(create=True, size=path_incs.nbytes)

    # Create numpy arrays from the shared memory buffers
    prevPath_shm_array = np.ndarray(prevPath_observations.shape, dtype=np.float64, buffer=prevPath_shm.buf)
    path_incs_shm_array = np.ndarray(path_incs.shape, dtype=np.float64, buffer=path_incs_shm.buf)

    # Copy the data into the shared memory arrays
    np.copyto(prevPath_shm_array, prevPath_observations)
    np.copyto(path_incs_shm_array, path_incs)

    # Ensure randomness across starmap calls
    master_seed = 42
    seed_seq = np.random.SeedSequence(master_seed)

    # Euler-Maruyama Scheme for Tracking Errors
    shape = prevPath_observations.shape
    M_tile = 32
    Nn_tile = 512000
    stable = True
    bw_idx = 30
    bw = bws[bw_idx, :]
    inv_H = np.diag(np.power(bw, -2))
    norm_const = 1 / np.sqrt((2. * np.pi) ** config.ndims * (1. / np.linalg.det(inv_H)))
    num_dhats = 10
    device = _get_device("cpu")

    # Upload once (float32)
    unif_is_drift_hats = np.zeros((Xshape, num_dhats, config.ts_dims))
    Xs = torch.as_tensor(Xs, dtype=torch.float32, device=device).contiguous()
    for k in (range(num_dhats)):
        is_ss_path_observations = is_path_observations[np.random.choice(is_idxs, size=num_paths, replace=False), :]
        is_prevPath_observations = is_ss_path_observations[:, 1:-1]
        is_path_incs = np.diff(is_ss_path_observations, axis=1)[:, 1:]
        is_prevPath_observations = torch.as_tensor(is_prevPath_observations, dtype=torch.float32, device=device).contiguous()
        is_path_incs = torch.as_tensor(is_path_incs, dtype=torch.float32, device=device).contiguous()
        # inv_H: prefer diagonal vector if possible
        inv_H_np = np.asarray(inv_H)
        if inv_H_np.ndim == 2 and np.allclose(inv_H_np, np.diag(np.diag(inv_H_np))):
            inv_H_vec = np.diag(inv_H_np).astype(np.float32)
            inv_H = torch.as_tensor(inv_H_vec, device=device)
        else:
            inv_H = torch.as_tensor(inv_H_np.astype(np.float32), device=device)

        unif_is_drift_hats[:, k, :]  =  IID_NW_multivar_estimator_gpu(
            is_prevPath_observations, is_path_incs, inv_H, float(norm_const),
            Xs, float(config.t1), float(config.t0),
            truncate=True, M_tile=M_tile, Nn_tile=Nn_tile, stable=stable
            ).numpy()
    return unif_is_drift_hats

In [28]:
remote_file_path = "/Users/marcos/Library/CloudStorage/OneDrive-ImperialCollegeLondon/StatML_CDT/Year2/DiffusionModels/" #project_config.ROOT_DIR+"data/"
remote_file_path

'/Users/marcos/Library/CloudStorage/OneDrive-ImperialCollegeLondon/StatML_CDT/Year2/DiffusionModels/'

In [29]:
def get_drift_files(config, root_dir, best_epoch):
    ts_type = f"{config.ndims}DLnz_".replace(".", "")
    include =  (f"_{1}NDT_{config.loss_factor}LFac_BetaMax{config.beta_max:.1e}_{round(config.forcing_const,3)}FConst").replace(
            ".", "")
    root_score_dir = root_dir + f"ExperimentResults/TSPM_Markovian/40DLnzChaosLessData/"
    driftoostrack_true_files = []
    driftoostrack_local_files = []
    for file in os.listdir(root_score_dir):
        if "MLP" in file and ts_type in file and include in file and ("_"+str(best_epoch)+"Nep") in file:
            if "DriftTrack" in file and "true" in file:
                driftoostrack_true_files.append(root_score_dir+file)
            elif "DriftTrack" in file and "global" in file:
                driftoostrack_local_files.append(root_score_dir+file)
    assert len(driftoostrack_true_files)>0, "No oos drift track files found"
    assert len(driftoostrack_local_files)>0, "No oos drift track files found"
    assert(len(driftoostrack_true_files) == len(driftoostrack_true_files))
    def extract_bw_drift_track_number(s):
        match = s.split("Nep_")[0].split("_")[-1]
        return int(match)
    driftoostrack_true_files = sorted(driftoostrack_true_files, key=extract_bw_drift_track_number)
    driftoostrack_local_files = sorted(driftoostrack_local_files, key=extract_bw_drift_track_number)
    Nepochs_track = [extract_bw_drift_track_number(f) for f in driftoostrack_true_files]
    return np.load(driftoostrack_true_files[0]), driftoostrack_local_files, Nepochs_track

In [30]:
true, _, _ = get_drift_files(config=config, root_dir=remote_file_path, best_epoch=best_epoch)
true = true.reshape(-1, config.ts_dims)

In [31]:
totN = 0
score_sse = 0
nad_sse = 0
for nsamp in tqdm(range(10)):
    true_nsamp = true[np.random.choice(np.arange(0,true.shape[0]),100),:]
    Xshape = true_nsamp.shape[0]
    Xs = true_nsamp
    true_drifts = true_drift(prev=Xs, num_paths=Xs.shape[0], config=config)
    es=config.max_diff_steps - 1
    DNonGauss = experiment_MLP_1D_drifts(good=good, es=es, Xs=Xs, config=config, onlyGauss=False)
    unif_is_drift_hats = run_Nadaraya(config=config, Xs=Xs)
    totN += true_nsamp.shape[0]
    score_sse += np.sum(np.sum(np.power(DNonGauss[:, 0, :, :].mean(axis=1) - true_drifts,2), axis=-1))
    nad_sse += np.sum(np.sum(np.power(unif_is_drift_hats.mean(axis=1) - true_drifts,2), axis=-1))

100%|██████████| 10/10 [01:32<00:00,  9.27s/it]


In [32]:
print(score_sse/totN, nad_sse/totN)

447.7020503815116 150.42973417928275
